In [38]:
from rcv_elections import rcvElections 
import matplotlib.pyplot as plt
import pandas as pd
import random
import itertools
from numpy.random import choice
import random
import sys
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
##generates a ballot list based on a PL distribution on all permutations specified by the second and third arguments (which are supposed to be dictionaries)
def pl(num_ballots, vote_portion_by_race, mean_support_by_race, cand_list):##borrowed from the ballot_generator in the GeneralRCV repo with some slight modifications
    ballots_list = []
    for race in vote_portion_by_race.keys():
        num_ballots_race = int(num_ballots*vote_portion_by_race[race])##this computes number of voters in this race/block
        cand_support_vec = list(mean_support_by_race[race].values())##creates the interval of probabilities for candidates supported by this block
        for j in range(num_ballots_race):
            (a,b,c,d,e) = choice(cand_list, len(cand_list), replace = False, p = cand_support_vec)
            ballots_list.append((a,b,c,d,e))
    return ballots_list
def pl8(num_ballots, vote_portion_by_race, mean_support_by_race, cand_list):##borrowed from the ballot_generator in the GeneralRCV repo with some slight modifications
    ballots_list = []
    for race in vote_portion_by_race.keys():
        num_ballots_race = int(num_ballots*vote_portion_by_race[race])##this computes number of voters in this race/block
        cand_support_vec = list(mean_support_by_race[race].values())##creates the interval of probabilities for candidates supported by this block
        for j in range(num_ballots_race):
            (a,b,c,d,e,f,g,h) = choice(cand_list, len(cand_list), replace = False, p = cand_support_vec)
            ballots_list.append((a,b,c,d,e,f,g,h))
    return ballots_list



def bt(num_ballots, vote_portion_by_race, mean_support_by_race, cand_list): ##generates a ballot list based on a BT distribution on all permutations specified by the second and third arguments (which are supposed to be dictionaries)
  weights=[]
  n=len(cand_list)
  k=0
  permutations = list(itertools.permutations(cand_list))
  for combo in permutations:##computes (inverse of) the constant of proportionality
    m=1
    for i in range(n):
      for j in range(i+1,n):
        l=0
        for race in vote_portion_by_race.keys():
          l = l+vote_portion_by_race[race]*(mean_support_by_race[race][combo[i]]/(mean_support_by_race[race][combo[i]]+mean_support_by_race[race][combo[j]]))
        if j!=i:
          m=m*l
    k=k+m

  for combo in permutations:
    prob=1
    for i in range(n):
      for j in range(i+1,n):
        l=0
        for race in vote_portion_by_race.keys():
          l = l+vote_portion_by_race[race]*(mean_support_by_race[race][combo[i]]/( mean_support_by_race[race][combo[i]]+mean_support_by_race[race][combo[j]]))
        prob=prob*l
    weights.append(prob/k)##we're giving each permutation of cand_list a weight

  x= choice(range(len(permutations)), num_ballots, replace=True, p = weights)
  return list(permutations[i] for i in x)

In [40]:
def ballotGen8(num_ballots, ratio_pl):
  cand_list = ['A','B','C','D','E','F','G','H']
  ratio_bt = 1 - ratio_pl
  num_bt = int(round(ratio_bt * num_ballots,0))
  num_pl= int(round(ratio_pl * num_ballots,0))
  mean_support_by_race = {'X':{'A':0.25, 'B':0.20, 'C':0.19, 'D':0.1, 'E':0.05, 'F': 0.03, 'G': 0.03, 'H': 0.15}, 'Y':{'A':.13, 'B':0.10, 'C':0.07, 'D':0.15, 'E': 0.05,'F': 0.2, 'G': 0.1, 'H': 0.2}}
  bt_list = bt(num_bt, {'X':0.5, 'Y':0.5}, mean_support_by_race, cand_list)
  pl_list = pl8(num_pl, {'X':0.5, 'Y':0.5}, mean_support_by_race, cand_list)
  election_list = bt_list + pl_list
  if num_ballots != len(election_list):
    under_count = num_ballots - len(election_list)
    more_ballots = bt(under_count, {'X':0.5, 'Y':0.5}, mean_support_by_race, cand_list)
    election_list = bt_list + pl_list + more_ballots
  return election_list

In [41]:
#test
test_ballot= ballotGen8(120, .3)
cand_list = ['A', 'B', 'C', 'D', 'E', 'F','G', 'H']
num_seats= 3

In [42]:
election = rcvElections(test_ballot, cand_list, num_seats)

election.rcv_run()

TypeError: cincinnati_transfer() missing 1 required positional argument: 'cutoff'